In [1]:
%matplotlib notebook

In [2]:
from scipy.stats import linregress
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [3]:
wine_df = pd.read_csv('Wine_data.csv', encoding="ISO-8859-1")

wine_df.head()

,Name,Rating,Winery,Area,Province,Country,Varietal,Vintage,Style,Wine Type,ID
0,IV,93.433789,Layor Wines,Nova Scotia,NaN,Canada,Vidal,2013,Dessert,White,55663e3e942360030000000e
1,Layor Gold,90.627673,Layor Wines,Nova Scotia,NaN,Canada,Vidal,2013,Dessert,White,55663e00942360030000000d
2,The Evangelist - Late Harvest Viognier,89.971973,Ascension Cellars,Central Coast,CA,USA,Viognier,2012,Dessert,White,5564efe1b4f0c10300000058
3,Soul Shaker,89.506316,Ascension Cellars,Central Coast,CA,USA,"Cabernet Sauvignon, Petit Verdot, Syrah, Merlot",2012,NaN,Red,5564efe1b4f0c1030000009c
4,"Dalliance ""GSMv""",89.421973,Ascension Cellars,Central Coast,CA,USA,"Grenache, Syrah, Mourvedre, Viognier",2011,NaN,Red,5564efe1b4f0c1030000009b


In [4]:
#Data Clanup

#Correcting United States to USA, Remove counties that don't exist

wine_df['Country'] = wine_df['Country'].replace({'United States': 'USA', 'California': 'USA', 'Franc': 'France', 'French': 'France'})

cleaned_df = wine_df.loc[(wine_df['Country'] != 'Blind') & (wine_df['Country'] != 'Macedonia')]

# cleaned2_df = cleaned_df.dropna(subset = ['Varietal'])

# len(cleaned_df) - len(cleaned_df.dropna(subset = ['Vintage']))

# cleaned_df['Vintage'].value_counts()

In [5]:
#Bar Chart of Countries with Average Rating

winery_count = cleaned_df.groupby('Country').count()['Winery']

wine_rating = cleaned_df.groupby('Country').mean()['Rating']

winery_count_df = pd.DataFrame({'Winery Count': winery_count})

wine_rating_df = pd.DataFrame({'Wine Rating': wine_rating})

top10_country = winery_count_df.nlargest(10, 'Winery Count') #10 largest by Winery Count

top10_rating = wine_rating_df.nlargest(10, 'Wine Rating')

merge_df = top10_country.merge(wine_rating_df, on = 'Country')

# top10_country.sort_values(by = 'Winery Count', ascending = False).plot(kind='bar', ylabel = 'Count of Wineries')

# wine_rating_df.plot(kind='line', secondary_y = True)

# plt.legend()

# plt.show()
merge_df.head(10)

,Winery Count,Wine Rating
Country,,
USA,4211,77.101990
Canada,2555,76.416850
France,1716,76.952722
Italy,1605,76.844544
Spain,588,76.671740
Australia,460,76.146005
Argentina,421,75.767816
Chile,311,75.813061
New Zealand,241,76.859121


In [6]:
cleaned_df.head()

,Name,Rating,Winery,Area,Province,Country,Varietal,Vintage,Style,Wine Type,ID
0,IV,93.433789,Layor Wines,Nova Scotia,NaN,Canada,Vidal,2013,Dessert,White,55663e3e942360030000000e
1,Layor Gold,90.627673,Layor Wines,Nova Scotia,NaN,Canada,Vidal,2013,Dessert,White,55663e00942360030000000d
2,The Evangelist - Late Harvest Viognier,89.971973,Ascension Cellars,Central Coast,CA,USA,Viognier,2012,Dessert,White,5564efe1b4f0c10300000058
3,Soul Shaker,89.506316,Ascension Cellars,Central Coast,CA,USA,"Cabernet Sauvignon, Petit Verdot, Syrah, Merlot",2012,NaN,Red,5564efe1b4f0c1030000009c
4,"Dalliance ""GSMv""",89.421973,Ascension Cellars,Central Coast,CA,USA,"Grenache, Syrah, Mourvedre, Viognier",2011,NaN,Red,5564efe1b4f0c1030000009b


In [8]:
# Vintage analysis

only_nv_df = cleaned_df.loc[(cleaned_df["Vintage"] == "nv")]
# only_nv_df

only_year_df = cleaned_df.loc[(cleaned_df["Vintage"] != "nv")]

# only_year_df["Vintage"] = int(only_year_df["Vintage"])

list_of_vintages = only_year_df["Vintage"]

only_year_df["Vintage"] = [int(i) for i in only_year_df["Vintage"]]

only_year_df["Vintage"]

after_05_df = only_year_df.loc[(only_year_df["Vintage"] >= 2010)]
after_05_df


after_05_grouped = after_05_df.groupby(["Country", "Vintage"])
vint_avg_rating = after_05_grouped.count()
most_rated = vint_avg_rating.loc[(vint_avg_rating["Rating"] > 100)]
most_rated

# top_countries = ["Canada", "France", "Italy", "USA"]

# create a bar graph that has 4 bars (top_countries) x axis is year (2010-2018) including nv and y axis is avg rating

<ipython-input-8-42a659510ce5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_year_df["Vintage"] = [int(i) for i in only_year_df["Vintage"]]


Name  Rating  Winery  Area  Province  Varietal  Style  \
Country   Vintage                                                          
Argentina 2013      101     101     101    96         6       101      1   
Canada    2011      143     143     143   141        57       140     18   
          2012      297     297     297   297       118       294     32   
          2013      328     328     328   327       103       327     23   
          2014      231     231     231   231        57       229     23   
          2015      207     207     207   207        34       207     16   
          2016      275     275     275   275        14       274     14   
          2017      355     355     355   353        19       351     16   
          2018      149     149     149   148        14       149      2   
France    2011      175     175     175   169        22       171      3   
          2012      201     201     201   190        29       198      2   
          2013      198     198     198   196        19       195      0   
          2014      163     163     163   159        23       163      2   
          2015      135     135     135   125        39       133      1   
Italy     2010      101     101     101    98         9       100      1   
          2011      153     153     153   147        15       151      4   
          2012      191     191     191   185        13       189      4   
          2013      242     242     242   235        22       241      4   
          2014      163     163     163   152        29       162      4   
          2015      122     122     122   117        42       122      7   
          2016      111     111     111    92        42       111     11   
USA       2010      212     212     212   211       116       211     19   
          2011      369     369     369   362       198       368     16   
          2012      723     723     723   710       304       721     29   
          2013      683     683     683   675       305       681     23   
          2014      422     422     422   418       191       422     11   
          2015      257     257     257   246       105       255      5   
          2016      247     247     247   237        60       247      3   
          2017      148     148     148   139        67       148      0   

                   Wine Type   ID  
Country   Vintage                  
Argentina 2013           101  101  
Canada    2011           143  143  
          2012           297  297  
          2013           328  328  
          2014           231  231  
          2015           207  207  
          2016           275  275  
          2017           355  355  
          2018           149  149  
France    2011           175  175  
          2012           201  201  
          2013           198  198  
          2014           163  163  
          2015           135  135  
Italy     2010           101  101  
          2011           153  153  
          2012           191  191  
          2013           242  242  
          2014           163  163  
          2015           122  122  
          2016           111  111  
USA       2010           212  212  
          2011           369  369  
          2012           723  723  
          2013           683  683  
          2014           422  422  
          2015           257  257  
          2016           247  247  
          2017           148  148

In [ ]:
#make a line graph
vint_avg_rating.plot.line()
